In [2]:
import {
  StationPayload,
  StationStatus,
  StationInfo,
  RegionPayload,
} from "./mod.ts";

const system_regions = await getJSON<RegionPayload>(
  "https://gbfs.bcycle.com/bcycle_santacruz/system_regions.json"
);

async function getJSON<T>(url: string): Promise<T> {
  const res = await fetch(url);
  return (await res.json()) as T;
}

const station_status = await getJSON<StationPayload<StationStatus>>(
  "https://gbfs.bcycle.com/bcycle_santacruz/station_status.json"
);

const station_info = await getJSON<StationPayload<StationInfo>>(
  "https://gbfs.bcycle.com/bcycle_santacruz/station_information.json"
);

const stationInfoMap = new Map(
  station_info.data.stations.map((s) => [s.station_id, s])
);

// Create a map of region IDs to region names for easy lookup
const regionNameMap = new Map(
  system_regions.data.regions.map((r) => [r.region_id, r.region_name])
);

let stations = station_status.data.stations;
stations = stations.map((station) => {
  return { ...station, ...stationInfoMap.get(station.station_id) };
});
stations[0];


{
  is_returning: 1,
  is_renting: 1,
  is_installed: 1,
  num_docks_available: 3,
  num_bikes_available: 0,
  last_reported: 1699049048,
  num_bikes_available_types: { electric: 0, smart: 0, classic: 0 },
  station_id: "bcycle_santacruz_7429",
  lon: -122.05488,
  lat: 36.98856,
  rental_uris: {
    ios: "https://www.bcycle.com/applink?system_id=bcycle_santacruz&station_id=bcycle_santacruz_7429&platform="... 3 more characters,
    android: "https://www.bcycle.com/applink?system_id=bcycle_santacruz&station_id=bcycle_santacruz_7429&platform="... 7 more characters
  },
  _bcycle_station_type: "3.0 Dock Station",
  region_id: "bcycle_santacruz_region_49",
  address: "The UCSC Village Lounge",
  name: "UCSC - The Village Lounge"
}

In [3]:
import React from "https://esm.sh/react";
import { renderToString } from "https://esm.sh/react-dom/server";

function renderToJupyter(el) {
  return Deno.jupyter.html([renderToString(el)]);
}


In [4]:
const groupedStations = stations.reduce((grouped, station) => {
  const { region_id } = station;
  if (!grouped[region_id]) {
    grouped[region_id] = [];
  }
  grouped[region_id].push(station);
  return grouped;
}, {});


In [6]:
import {
  StationPayload,
  StationStatus,
  StationInfo,
  RegionPayload,
} from "./mod.ts";

function Station(props: StationStatus & StationInfo) {
  const {
    num_bikes_available,
    num_docks_available,
    name,
    address,
    rental_uris,
    _bcycle_station_type,
    region_id,
  } = props;
  const backgroundColor = num_bikes_available > 0 ? "#00ff55" : "#ff5500";

  return (
    <div
      style={{
        position: "relative",
        margin: "10px",
        padding: "10px",
        border: "1px solid #ddd",
        borderRadius: "8px",
        display: "inline-block",
        width: "300px",
      }}
    >
      <h3 style={{ margin: "0 0 10px 0" }}>{name}</h3>
      <h4 style={{ margin: "0" }}>{address}</h4>
      <div
        style={{
          backgroundColor,
          padding: "10px",
          color: "white",
          fontWeight: "bold",
          textAlign: "center",
          borderRadius: "4px",
        }}
      >
        Electric Bikes Available: {num_bikes_available}
      </div>
      <div style={{ marginTop: "10px" }}>
        <div>Docks Available: {num_docks_available}</div>
        <div>Type: {_bcycle_station_type}</div>
        <div>
          Rent on iOS:{" "}
          <a href={rental_uris.ios} target="_blank">
            Link
          </a>
        </div>
        <div>
          Rent on Android:{" "}
          <a href={rental_uris.android} target="_blank">
            Link
          </a>
        </div>
      </div>
    </div>
  );
}

function RegionGroup({ regionId, stations }) {
  const regionName = regionNameMap.get(regionId) || "Unknown Region";

  const last_reported = stations[0].last_reported;

  const lastReportedDate = new Date(last_reported * 1000).toLocaleString();
  return (
    <div>
      <h2>{regionName}</h2>
      <div
        style={{
          position: "absolute",
          top: "10px",
          right: "10px",
          fontSize: "0.8em",
        }}
      >
        Last Reported: {lastReportedDate}
      </div>
      <div
        style={{ display: "flex", flexWrap: "wrap", justifyContent: "center" }}
      >
        {stations.map((station) => (
          <Station key={station.station_id} {...station} />
        ))}
      </div>
    </div>
  );
}

renderToJupyter(
  <div>
    {Object.entries(groupedStations).map(([regionId, stations]) => (
      <RegionGroup key={regionId} regionId={regionId} stations={stations} />
    ))}
  </div>
);


UCSC Last Reported: <!-- -->11/3/2023, 3:04:08 PM UCSC - The Village Lounge The UCSC Village Lounge Electric Bikes Available: <!-- -->0 Docks Available: <!-- -->3 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC - East Remote Lot Interior Parking Lot 104 Electric Bikes Available: <!-- -->3 Docks Available: <!-- -->2 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC - Theatre Arts 453 Kerr Rd Electric Bikes Available: <!-- -->0 Docks Available: <!-- -->24 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC - Family Student Housing Transit Stop Heller Dr Electric Bikes Available: <!-- -->1 Docks Available: <!-- -->3 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC - Science & Engineering Library 580 Red Hill Rd Electric Bikes Available: <!-- -->0 Docks Available: <!-- -->18 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC - Westside Research Park 2300 Delaware Ave Electric Bikes Available: <!-- -->7 Docks Available: <!-- -->1 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC - Coastal Biology Building McAllister Way Electric Bikes Available: <!-- -->2 Docks Available: <!-- -->5 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC - Perk Coffee/Physical Sciences Between Steinhart Way and Red Hill Rd Electric Bikes Available: <!-- -->0 Docks Available: <!-- -->16 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC - Baskin Engineering McLaughlin Dr Electric Bikes Available: <!-- -->0 Docks Available: <!-- -->13 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC - Seymour Marine Discovery Center 100 McAllister Way Electric Bikes Available: <!-- -->3 Docks Available: <!-- -->7 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC – Humanities Cowell & Stevenson Road Electric Bikes Available: <!-- -->0 Docks Available: <!-- -->9 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC – East Field House Main Entrance East Field Service Rd Electric Bikes Available: <!-- -->0 Docks Available: <!-- -->17 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC - Graduate Student Housing Heller Dr. & McLaughlin Dr. Electric Bikes Available: <!-- -->0 Docks Available: <!-- -->23 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC - Hahn Parking Lot 101 Electric Bikes Available: <!-- -->0 Docks Available: <!-- -->4 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC – I-Lounge Porter-Kresge Road Electric Bikes Available: <!-- -->1 Docks Available: <!-- -->7 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC - Merrill College Merrill Road/Circle Electric Bikes Available: <!-- -->0 Docks Available: <!-- -->12 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC - West Remote Parking Lot Parking Lot 127 Electric Bikes Available: <!-- -->4 Docks Available: <!-- -->2 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC - Oakes College Oakes Rd/Circle Electric Bikes Available: <!-- -->1 Docks Available: <!-- -->11 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC – Rachel Carson College Parking Lot 146 Electric Bikes Available: <!-- -->6 Docks Available: <!-- -->9 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!-- --> Link UCSC – Porter College Quad Porter College House B Electric Bikes Available: <!-- -->0 Docks Available: <!-- -->12 Type: <!-- -->3.0 Dock Station Rent on iOS:<!-- --> Link Rent on Android:<!--